# Neural Style Transfer

> Neural style transfer is an optimization technique where two images—a *content* image and a *style reference* image (such as an artwork by a famous painter)—are blend together to create an output image which would look like the content image, but “painted” in the style of the style reference image.



In [0]:
# Importing libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime

%matplotlib inline

In [0]:
# Hyper-parameters
CONTENT_WEIGHT = 1e3
STYLE_WEIGHT = 1e1
EPOCHS = 1000

# Utility Functions

In [0]:
def show_result(image, save_fig=False):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)
  
  fig = plt.figure() 
  fig.figsize=(15,15)
  plt.title("Styled Image")
  plt.axis("off")
  plt.imshow(image)
  
  if save_fig:
    fig.savefig(f"styled_image_{datetime.now()}.png")

In [0]:
def clip_img(img, min_val=0.0, max_val=1.0):
  return tf.clip_by_value(img, min_val, max_val, name="clipping_img")

In [0]:
def scale_img(img, max_size=512):
  shape = tf.cast(img.shape[:-1], dtype=tf.float32)
  max_dim = max(shape)
  scale = tf.constant(max_size / max_dim, dtype=tf.float32)
  new_shape = tf.cast(shape * scale, dtype=tf.int32)
  resized_img = tf.image.resize(img, new_shape, name="resizing_img")
  return resized_img[tf.newaxis, ...]

In [0]:
def prepare_img(img_loc, img_name="input_img"):
  raw_img = tf.io.read_file(img_loc)
  img = tf.io.decode_image(raw_img, channels=3, dtype=tf.float32, 
                           name= img_name)
  scaled_img = scale_img(img)
  return scaled_img

In [0]:
def create_model(layer_names):
    # Loading pre-trained VGG19 network
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    output_layers = [vgg.get_layer(name).output for name in layer_names]
    return tf.keras.Model(vgg.input, output_layers)

# Model Definition & Losses

In [0]:
def loss_cal(outputs, content_targets, style_targets):
  content_outputs = outputs["content_map"]
  style_outputs = outputs["style_map"]
  
  content_loss = [tf.reduce_mean((content_targets[layer_name] - content_outputs[layer_name])**2)
                                 for layer_name in content_outputs.keys()]
  content_loss = tf.reduce_mean(content_loss, name="content_loss")
  
  style_loss = [tf.reduce_mean((style_targets[layer_name] - style_outputs[layer_name])**2)
                                 for layer_name in style_outputs.keys()]
  style_loss = tf.reduce_mean(style_loss, name="style_loss")

  total_loss = CONTENT_WEIGHT * content_loss + STYLE_WEIGHT * style_loss
  return total_loss

In [0]:
def train_step(image, model, content_targets, style_targets, sum_writer, epoch):
  with tf.GradientTape() as tape:
    outputs = model(image)
    loss = loss_cal(outputs, content_targets, style_targets)

  gradients = tape.gradient(loss, image)
  model.optimizer.apply_gradients([(gradients, image)])
  image.assign(clip_img(image))
  
  with sum_writer.as_default():
    tf.summary.scalar('Loss', loss, step=epoch)

In [0]:
class NSTModel(tf.keras.Model):

  def __init__(self, content_layers, style_layers):
    super(NSTModel, self).__init__()
    self.optimizer = tf.optimizers.Adam(learning_rate=0.001, beta_1=0.99, 
                                        epsilon=1e-1)
    self.model = create_model(content_layers + style_layers)
    self.preprocessor = tf.keras.applications.vgg19.preprocess_input
    self.content_layers = content_layers
    self.style_layers = style_layers
    self.n_content_layers = len(content_layers)

  def call(self, inputs):
    inputs = inputs * 255.0
    processed_inputs = self.preprocessor(inputs)
    outputs = self.model(processed_inputs)
    content_outputs, style_outputs = outputs[:self.n_content_layers], \
                                      outputs[self.n_content_layers:]
    style_corr_values = [self._gram_matrix(feature_map) 
                        for feature_map in style_outputs]
    content_map = {layer_name: layer_output for layer_name, layer_output 
                   in zip(self.content_layers, content_outputs)}
    style_map = {layer_name: layer_output for layer_name, layer_output
                 in zip(self.style_layers, style_corr_values)}
    return {"content_map": content_map, "style_map": style_map}

  def _gram_matrix(self, feature_map):
    b, h, w, c = feature_map.shape
    normalization_factor = tf.cast(h * w, dtype=tf.float32)
    gram_matrix = tf.linalg.einsum('bijc,bijd->bcd', feature_map, feature_map, 
                                  name="gram_matrix")
    return gram_matrix / normalization_factor

# Content & Style Layers

In [0]:
# Choosing our content layers
content_layers = ["block5_conv2"]

# Choosing out style layers
style_layers = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", 
                "block5_conv1"]

# Style Transfer Begins

In [0]:
# Creating object for our model
model = NSTModel(content_layers, style_layers)

In [0]:
# Loading our content image
content_img = prepare_img("bhai.jpeg")

# Loading our style image
style_img = prepare_img("style.jpg")

# Preparing our output image
output_img = tf.Variable(content_img)

In [0]:
# Preparing our target content values
content_targets = model(content_img)["content_map"]

# Preparing our target style values
style_targets = model(style_img)["style_map"]

In [0]:
# Preparing tensorboad for visualization
log_dir = "logs/"
summary_writer = tf.summary.create_file_writer(
    log_dir + "fit/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

%load_ext tensorboard
%tensorboard --logdir {log_dir}

In [0]:
# Running gradient descent on output image
for epoch in range(1, EPOCHS+1):
  train_step(output_img, model, content_targets, style_targets, 
                    summary_writer, epoch)
  if epoch % 200 == 0:
    show_result(output_img, True)